# SUPERT 

Example code for using the supert metrics

- Paper: https://arxiv.org/abs/2005.03724
- Git-Project: https://github.com/danieldeutsch/SUPERT

In [1]:
import sys
sys.path.append("SUPERT")

In [2]:
# to get the reader working
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dwarakvittal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dwarakvittal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from ref_free_metrics.supert import Supert
from utils.data_reader import CorpusReader

In [4]:
# read docs and summaries
reader = CorpusReader('SUPERT/data/topic_1')
source_docs = reader()
summaries = reader.readSummaries() 

# Evaluate Summaries

In [5]:
# compute the Supert scores
supert = Supert(docs=source_docs) 
#supert.load_documents(source_docs)
scores = supert(summaries)

In [8]:
print(len(summaries))
print(summaries)
print(scores)

2
(['summary1.txt', 'summary2.txt', 'summary3.txt', 'summary4.txt', 'summary5.txt'], ['Juan Miguel Alvarez, charged with murder with special circumstances in the deaths of 11 Metrolink passengers, slashed and stabbed himself after seeing the horrific train crash he caused, sources close to the investigation said Thursday. Alvarez, 25, despondent over the breakup of his marriage, had planned to kill himself when he drove his green Jeep Grand Cherokee in the path of a Metrolink train, officials said, but he bolted from the vehicle at the last moment. As he watched, southbound Train No.\n', 'Until Wednesday, Juan Manuel Alvarez was living the average life of an obscure and troubled man.It had to be the worst multicasualty incident Ive been to, Los Angeles Fire Capt. Rick Godinez said.David Morrison, 47, an attorney, was heading to downtown Los Angeles on his regular morning commute. With his tire, apparently caught between the tracks, Mr. Alvarez jumped out of the Jeep and ran. But, Mrs. 

### RL Class from other py-script

In [9]:
from ref_free_metrics.supert import Supert
from summariser.ngram_vector.vector_generator import Vectoriser
from summariser.deep_td import DeepTDAgent as RLAgent
from utils.data_reader import CorpusReader
from utils.evaluator import evaluate_summary_rouge, add_result

class RLSummarizer():
    def __init__(self,reward_func, reward_strict=5.,rl_strict=5.,train_episode=5000, base_length=200, sample_summ_num=5000, gpu=False):
        self.reward_func = reward_func
        self.reward_strict = reward_strict
        self.rl_strict = rl_strict
        self.train_episode = train_episode
        self.base_length = base_length
        self.sample_summ_num = sample_summ_num
        self.gpu = gpu

    def get_sample_summaries(self, docs, summ_max_len=100):
        vec = Vectoriser(docs,summ_max_len)
        summary_list = vec.sample_random_summaries(self.sample_summ_num)
        rewards = self.reward_func(summary_list)
        assert len(summary_list) == len(rewards)
        return summary_list, rewards

    def summarize(self, docs, summ_max_len=100):
        # generate sample summaries for memory replay
        summaries, rewards = self.get_sample_summaries(docs, summ_max_len)
        vec = Vectoriser(docs,base=self.base_length)
        rl_agent = RLAgent(vec, summaries, strict_para=self.rl_strict, train_round=self.train_episode, gpu=self.gpu)
        summary = rl_agent(rewards)
        return summary

# Create Summaries with RL

In [10]:
# generate summaries using reinforcement learning, with supert as reward function
supert = Supert(source_docs)
rl_summarizer = RLSummarizer(reward_func = supert, gpu = False)
summary = rl_summarizer.summarize(source_docs, summ_max_len=100)

# print out the generated summary
print(summary)

neural-rl training episodes: 100%|██████████| 5000/5000 [00:39<00:00, 127.07it/s]


Distraught and remorseful, Alvarez told police he had left the vehicle and watched the derailment, Adams said. ``I think his intent at that time was to take his own life, but (he) changed his mind prior to the train actually striking the vehicle.'' At 6:02 a.m., the three-car Train 100, headed from Moorpark to Union Station, with the locomotive in the rear, crashed into the SUV. Until Wednesday, Juan Manuel Alvarez was living the average life of an obscure and troubled man. Adams identified the suspect as Juan Manuel Alvarez, 25, of Compton, adding that he had attempted suicide before.
